In [4]:
import pandas as pd

# Load datasets
matches = pd.read_csv('../data/raw/matches.csv')
deliveries = pd.read_csv('../data/raw/deliveries.csv')

print("IPL Data Loaded:")
print(f"✅ Matches: {matches.shape}")
print(f"✅ Deliveries: {deliveries.shape}")

IPL Data Loaded:
✅ Matches: (1095, 20)
✅ Deliveries: (260920, 17)
